In [ ]:
with open('/content/drive/MyDrive/legal_vocab_final_sorted_2.txt') as f:
    legal_vocab_list = [line.rstrip('\n') for line in f]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
with open('/content/drive/MyDrive/supremecourtdataset_816casestotal_oldandnew.json', encoding="utf-8") as data_file:    
  data = json.load(data_file)

In [ ]:
opinions = []
for scotus_case in data['cases']:
    opinion = scotus_case['opinion']
    opinions.append(opinion)

In [ ]:
syllabi = []
for scotus_case in data['cases']:
    syllabus = scotus_case['syllabus']
    syllabi.append(syllabus)

In [ ]:
case_ids = []
for scotus_case in data['cases']:
    id = scotus_case['id']
    case_ids.append(id)

In [ ]:
legal_term_dict = {}

In [ ]:
legal_term_dict

{}

In [ ]:
for term in legal_vocab_list:
  legal_term_dict[term] = 0
  

for term in legal_vocab_list: 
  for opinion in opinions:
    if term.lower() in opinion.lower():
      legal_term_dict[term] += 1

In [ ]:
import numpy as np

keys = np.array(list(legal_term_dict.keys()))
values = np.array(list(legal_term_dict.values()))

terms_in_dataset = list(keys[values!=0])

In [ ]:
print(len(terms_in_dataset))

9033


In [ ]:
!pip install eyecite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.4/801.4 KB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 KB 14.9 MB/s eta 0:00:00


In [ ]:
from eyecite import get_citations, resolve_citations

text = 'first citation: 1 U.S. 12. second citation: 2 F.3d 2. third citation: Id.'

In [ ]:
found_citations = get_citations(text)
resolved_citations = resolve_citations(found_citations)

In [ ]:
resolved_citations.values()

dict_values([[FullCaseCitation('1 U.S. 12', groups={'volume': '1', 'reporter': 'U.S.', 'page': '12'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court='scotus', plaintiff=None, defendant=None, extra=None))], [FullCaseCitation('2 F.3d 2', groups={'volume': '2', 'reporter': 'F.3d', 'page': '2'}, metadata=FullCaseCitation.Metadata(parenthetical=None, pin_cite=None, year=None, court=None, plaintiff=None, defendant=None, extra=None)), IdCitation('Id.', metadata=IdCitation.Metadata(parenthetical=None, pin_cite=None))]])

In [ ]:
from eyecite.tokenizers import default_tokenizer

In [ ]:
matched_text = []

In [ ]:
total_found_citations = []

for opinion in opinions:
  found_citations = get_citations(opinion)
  total_found_citations.append(found_citations)

In [ ]:
for i in total_found_citations:
  for j in i:
    actual_citation = j.matched_text()
    matched_text.append(actual_citation)

In [ ]:
len(matched_text)

11388

In [ ]:
citations_in_dataset = list(set(matched_text))

In [ ]:
len(citations_in_dataset)

6050

In [ ]:
output_file = open('/content/drive/MyDrive/citations_in_scotus_dataset.txt', 'w')

for citation in citations_in_dataset:
    output_file.write(citation + '\n')

In [ ]:
with open('/content/drive/MyDrive/legal_terms_in_scotus_dataset.txt') as f:
    legal_terms_in_dataset = [line.rstrip('\n') for line in f]

In [ ]:
with open('/content/drive/MyDrive/citations_in_scotus_dataset.txt') as f:
    case_citations_in_dataset = [line.rstrip('\n') for line in f]

In [ ]:
!pip install huggingface
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import *

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('nlpaueb/legal-bert-base-uncased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
custom_model = AutoModel.from_pretrained('nlpaueb/legal-bert-base-uncased', config=custom_config)

from summarizer import Summarizer

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/config.json
Model config BertConfig {
  "_name_or_path": "nlpaueb/legal-bert-base-uncased",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_pro

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--nlpaueb--legal-bert-base-uncased/snapshots/15b570cbf88259610b082a167dacc190124f60f6/pytorch_model.bin
Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing 

In [ ]:
custom_tokenizer.add_tokens(legal_vocab_list)
custom_tokenizer.add_tokens(case_citation_list)

NameError: ignored

In [ ]:
custom_model.resize_token_embeddings(len(custom_tokenizer))

NameError: ignored

In [ ]:
model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [ ]:
opinions_extractive_summaries = []
# import itertools

for opinion in opinions[:100]:
  opinion_extractive_summary = model(opinion, num_sentences=25)
  opinions_extractive_summaries.append(opinion_extractive_summary)

for opinion in opinions[100:]:
  opinion_extractive_summary = model(opinion, ratio=0.5)
  opinions_extractive_summaries.append(opinion_extractive_summary)

In [ ]:
print(opinions_extractive_summaries[400])

Mr. Justice Brown
delivered the opinion of the court. Objection was made by the government to the allowance of the following items:
• 1. “ In the case of United States v. Ewing, ante, 142, we held that where the local practice required a magistrate to reduce the examination of the complaining wit.nesses to. writing, an allowance for drawing the complaint, *166as “for taking and.certifying depositions to file,” was a proper charge under Rev. Stat. § The objection to this item is therefore overruled. The fourth series of items relates to charges in connection with the recognizances of defendants for examination. " There is no valid objection to the allowance *167for the oaths of sureties and the jurats .to such oaths. It is true that the talcing of recognizance or bail for appearance is primarily for the benefit of the defendant, and in civil cases it is usual to require the costs of entering into such recognizances to be paid by the defendant or other person offering himself as surety. 

In [ ]:
syllabi_extractive_summaries = []
# import itertools

for syllabus in syllabi[:100]:
  syllabus_extractive_summary = model(syllabus, num_sentences=10)
  syllabi_extractive_summaries.append(syllabus_extractive_summary)

for syllabus in syllabi[100:]:
  syllabus_extractive_summary = model(syllabus, ratio=0.5)
  syllabi_extractive_summaries.append(syllabus_extractive_summary)

In [ ]:
supremecourtdataset_816cases_extracted = []

In [ ]:
len(case_ids)

816

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 5.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
for i in range(len(case_ids)):
  case_extracted = dict({'id': case_ids[i], 'syllabus': syllabi_extractive_summaries[i], 'opinion': opinions_extractive_summaries[i]})
  supremecourtdataset_816cases_extracted.append(case_extracted)

In [ ]:
with open("/content/drive/MyDrive/supremecourtdataset_816totalcases_extracted.json", "w") as outfile:
  json.dump(supremecourtdataset_816cases_extracted, outfile)

In [ ]:
import json
with open('/content/drive/MyDrive/supremecourtdataset_816totalcases_extracted.json', encoding="utf-8") as data_file:    
  data = json.load(data_file)

In [ ]:
opinions = []
for scotus_case in data['cases']:
    opinion = scotus_case['opinion']
    opinions.append(opinion)

syllabi = []
for scotus_case in data['cases']:
    syllabus = scotus_case['syllabus']
    syllabi.append(syllabus)

In [ ]:
from transformers import EncoderDecoderModel, BertGenerationEncoder, BertGenerationDecoder, BertTokenizer, BartTokenizerFast, BartForConditionalGeneration

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('json', data_files='/content/drive/MyDrive/supremecourtdataset_816totalcases_extracted.json', field='cases', split='train')

In [ ]:
dataset = dataset.train_test_split(test_size=.2)
dataset = dataset.shuffle(seed=42)

In [ ]:
!pip install wandb -qU

In [ ]:
import wandb

In [ ]:
wandb.login()

wandb: Currently logged in as: knarasi1 (winter2023projects). Use `wandb login --relogin` to force relogin


True

In [ ]:
tokenizer = BartTokenizerFast.from_pretrained('bheshaj/bart-large-billsum-epochs20')

In [ ]:
tokenizer.add_tokens(legal_terms_in_dataset)

7868

In [ ]:
tokenizer.add_tokens(case_citations_in_dataset)

6048

In [ ]:
# from transformers import BertGenerationConfig, EncoderDecoderConfig

model = BartForConditionalGeneration.from_pretrained('bheshaj/bart-large-billsum-epochs20')

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(64181, 1024)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["opinion"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["syllabus"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/652 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_30epochs",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    gradient_accumulation_steps=8,
    report_to="wandb",
    logging_steps=10
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, syllabus, opinion. If id, syllabus, opinion are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 652
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 300
  Number of trainable parameters = 420541440
Automatic Weights & Biases logging enabled, to disable set os.environ["W

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,6.546000,4.643119,0.109700,0.025500,0.092200,0.092200,20.000000
1,4.785500,4.320249,0.127700,0.031000,0.103500,0.103600,19.884100
2,4.487800,4.181683,0.140300,0.040200,0.114900,0.114900,19.780500
3,4.342300,4.107414,0.140900,0.039900,0.113600,0.113600,19.652400
4,4.219200,4.030790,0.137900,0.038200,0.112100,0.112100,19.591500
5,4.118100,3.972375,0.132600,0.038400,0.105700,0.105900,19.323200
6,4.015400,3.905478,0.135100,0.042000,0.110600,0.110500,19.365900
7,3.920900,3.853427,0.131600,0.038100,0.107700,0.107300,19.384100
8,3.826300,3.809065,0.131100,0.036800,0.105400,0.105500,19.445100
9,3.743400,3.772098,0.114000,0.033700,0.090200,0.090100,18.067100


Streaming output truncated to the last 5000 lines.
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, syllabus, opinion. If id, syllabus, opinion are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 164
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true

TrainOutput(global_step=300, training_loss=3.6577638498942058, metrics={'train_runtime': 1355.7255, 'train_samples_per_second': 14.428, 'train_steps_per_second': 0.221, 'total_flos': 4.058297848214323e+16, 'train_loss': 3.6577638498942058, 'epoch': 29.98})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3
Configuration saved in BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/config.json
Configuration saved in BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/generation_config.json
Model weights saved in BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/pytorch_model.bin
tokenizer config file saved in BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/tokenizer_config.json
Special tokens file saved in BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/1.57G [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.62k/3.62k [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/knarasi1/BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3
   263c791..c198930  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/knarasi1/BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3
   263c791..c198930  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 0.1251}]}
To https://huggingface.co/knarasi1/BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3
   c198930..394144a  main -> main

   c198930..394144a  main -> main



'https://huggingface.co/knarasi1/BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3/commit/c198930f652412524cff092502ad822c6191171e'

In [ ]:
trainer.save_model("/content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/")

Saving model checkpoint to /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/
Configuration saved in /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/config.json
Configuration saved in /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/generation_config.json
Model weights saved in /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/special_tokens_map.json
Saving model checkpoint to BART_pretrained_on_billsum_finetuned_on_small_SCOTUS_extracted_dataset_3
Configuration saved in BART_pretrained_on_bi

In [ ]:
predict_results = trainer.predict(
    tokenized_dataset["test"], metric_key_prefix="predict", max_length=512
)

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: opinion, id, syllabus. If opinion, id, syllabus are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 164
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

In [ ]:
prediction_metrics = predict_results.metrics

In [ ]:
trainer.log_metrics("predict", prediction_metrics)
trainer.save_metrics("predict", prediction_metrics)

***** predict metrics *****
  predict_gen_len            =   113.1768
  predict_loss               =     3.5827
  predict_rouge1             =     0.2975
  predict_rouge2             =     0.0845
  predict_rougeL             =     0.1831
  predict_rougeLsum          =     0.1832
  predict_runtime            = 0:02:51.50
  predict_samples_per_second =      0.956
  predict_steps_per_second   =      0.122


In [ ]:
import os

predictions = tokenizer.batch_decode(
    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
predictions = [pred.strip() for pred in predictions]
output_prediction_file = os.path.join("/content/drive/MyDrive/BART_pretrained_on_billsum_finetuned_on_SCOTUS_extracted_dataset_6th_version/", "generated_predictions.txt")
with open(output_prediction_file, "w") as writer:
    writer.write("\n\n".join(predictions))